# Week 9: Hallucination Detection and Verification

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Digital-AI-Finance/agentic-artificial-intelligence/blob/main/L09_Hallucination_Prevention/L09_Verification.ipynb)

Implementing Chain-of-Verification and claim decomposition for hallucination detection.

In [ ]:
import sys
if 'google.colab' in sys.modules:
    !pip install -q langchain-openai python-dotenv
    from google.colab import userdata
    import os
    os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

In [ ]:
from typing import List, Dict
from dataclasses import dataclass
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

load_dotenv()
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
print("Ready")

## 1. Claim Decomposition

In [ ]:
def decompose_claims(text: str) -> List[str]:
    """Decompose text into atomic claims."""
    prompt = f"""Decompose this text into atomic, verifiable claims.
Each claim should be a single fact that can be verified independently.

Text: {text}

Return one claim per line, numbered."""
    
    response = llm.invoke(prompt).content
    claims = [line.strip() for line in response.split('\n') if line.strip()]
    claims = [c.lstrip('0123456789.-) ') for c in claims]
    return [c for c in claims if c]

# Test
test_text = """Albert Einstein developed the theory of relativity in 1905. 
He was born in Germany and later became a US citizen. He won the Nobel Prize in Physics."""

claims = decompose_claims(test_text)
for i, claim in enumerate(claims, 1):
    print(f"{i}. {claim}")

## 2. Generate Verification Questions

In [ ]:
def generate_verification_questions(claim: str) -> List[str]:
    """Generate questions to verify a claim."""
    prompt = f"""Generate 2-3 questions that, if answered correctly, would verify this claim.

Claim: {claim}

Return one question per line."""
    
    response = llm.invoke(prompt).content
    questions = [q.strip().lstrip('0123456789.-) ') for q in response.split('\n') if q.strip()]
    return [q for q in questions if q and '?' in q]

# Test
test_claim = "Albert Einstein developed the theory of relativity in 1905"
questions = generate_verification_questions(test_claim)
print(f"Claim: {test_claim}")
print("Verification questions:")
for q in questions:
    print(f"  - {q}")

## 3. Independent Verification

In [ ]:
@dataclass
class VerificationResult:
    claim: str
    is_verified: bool
    confidence: float
    reasoning: str

def verify_claim_independently(claim: str, questions: List[str]) -> VerificationResult:
    """Verify claim by answering questions independently."""
    answers = []
    for q in questions:
        response = llm.invoke(f"Answer factually: {q}").content
        answers.append((q, response))
    
    # Check consistency
    check_prompt = f"""Based on these Q&A pairs, is the claim verified?

Claim: {claim}

Q&A:
{chr(10).join([f'Q: {q} A: {a}' for q, a in answers])}

Respond with:
VERIFIED: [YES/NO]
CONFIDENCE: [0.0-1.0]
REASONING: [explanation]"""
    
    response = llm.invoke(check_prompt).content
    
    is_verified = "YES" in response.split("VERIFIED:")[1].split("\n")[0].upper() if "VERIFIED:" in response else False
    try:
        confidence = float(response.split("CONFIDENCE:")[1].split("\n")[0].strip())
    except:
        confidence = 0.5
    reasoning = response.split("REASONING:")[1].strip() if "REASONING:" in response else ""
    
    return VerificationResult(claim, is_verified, confidence, reasoning)

# Test
result = verify_claim_independently(test_claim, questions)
print(f"Verified: {result.is_verified}")
print(f"Confidence: {result.confidence}")
print(f"Reasoning: {result.reasoning[:200]}...")

## 4. Full Chain-of-Verification Pipeline

In [ ]:
def chain_of_verification(text: str) -> Dict:
    """Complete Chain-of-Verification pipeline."""
    print("Step 1: Decomposing claims...")
    claims = decompose_claims(text)
    
    results = []
    for i, claim in enumerate(claims[:5], 1):  # Limit for demo
        print(f"Step 2-3: Verifying claim {i}/{len(claims[:5])}...")
        questions = generate_verification_questions(claim)
        result = verify_claim_independently(claim, questions)
        results.append(result)
    
    verified_count = sum(1 for r in results if r.is_verified)
    avg_confidence = sum(r.confidence for r in results) / len(results) if results else 0
    
    return {
        "total_claims": len(claims),
        "verified_claims": verified_count,
        "average_confidence": avg_confidence,
        "results": results
    }

# Test with potential hallucination
test_text = """The Eiffel Tower was built in 1889 for the World's Fair.
It is located in Berlin, Germany. Gustave Eiffel designed it."""

output = chain_of_verification(test_text)
print(f"\nResults: {output['verified_claims']}/{output['total_claims']} verified")
print(f"Average confidence: {output['average_confidence']:.2f}")
for r in output['results']:
    status = 'Y' if r.is_verified else 'N'
    print(f"  [{status}] {r.claim[:60]}...")

## Summary

Implemented Chain-of-Verification:
1. **Claim Decomposition**: Break text into atomic facts
2. **Question Generation**: Create verification questions
3. **Independent Verification**: Answer without seeing original claim
4. **Consistency Check**: Compare answers to original claim